# Hosting Models on SageMaker and Automate the Workflow

In this module you will:
- Host a pretrained SKLearn model on SageMaker
- Enable autoscaling on your endpoint 
- Monitor your model
- Perform hyperparameter tuning
- Redploy a new model to the endpoint
- Automate the pipeline using the notebook runner toolkit

Let's get started! 

---

### 1. Access your model artifact
First, you should see a `model.tar.gz` file in this repository. Let's get that in your S3 bucket.

In [8]:
import sagemaker
import os

sess = sagemaker.Session()

# sagemaker will check to make sure this is a valid tar.gz object
local_model_file = 'model.tar.gz'

bucket = sess.default_bucket()

prefix = 'model-hosting'

s3_path = 's3://{}/{}/'.format(bucket, prefix)

msg = 'aws s3 cp {} {}'.format(local_model_file, s3_path)

os.system(msg)

0

### 2. Load your pretrained model artifact into SageMaker
Now, we know that this model was trained using the SKLearn container within SageMaker. All we need to do get this into a SageMaker-managed endpoint is set it up as a model. Let's do that here!

In [10]:
model_data = '{}{}'.format(s3_path, local_model_file)
print (model_data)

s3://sagemaker-us-east-1-181880743555/model-hosting/model.tar.gz


In [91]:
%%writefile train.py

import argparse
import pandas as pd
import numpy as np
import os

from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib

def model_fn(model_dir):
    """Deserialized and return fitted model

    Note that this should have the same name as the serialized model in the main method
    """
    regr = joblib.load(os.path.join(model_dir, "model.joblib"))
    return regr

def predict_fn(input_data, model):
    '''return the class and the probability of the class'''
    prediction = model.predict(input_data)
    pred_prob = model.predict_proba(input_data) #a numpy array
    return np.array(pred_prob)

def parse_args():
    
    # Hyperparameters are described here. In this simple example we are just including one hyperparameter.

    parser = argparse.ArgumentParser()
    
    parser.add_argument('--max_leaf_nodes', type=int, default=-1)

   # Sagemaker specific arguments. Defaults are set in the environment variables.
    parser.add_argument('--output-data-dir', type=str, default=os.environ['SM_OUTPUT_DATA_DIR'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])
    parser.add_argument('--test', type=str, default = os.environ['SM_CHANNEL_TEST'])
    
    # hyperparameters for tuning
    parser.add_argument('--batch-size', type=int, default=256)
    parser.add_argument('--lr', type=float, default = 0.001)
    
    args = parser.parse_args()
    
    return args

def train(args):
    
   # Take the set of files and read them all into a single pandas dataframe
    train_data=pd.read_csv(os.path.join(args.train, 'train_set.csv'), engine='python')

    # labels are in the first column
    train_y = train_data['truth']
    train_X = train_data[train_data.columns[1:len(train_data)]]

    # Now use scikit-learn's MLP Classifier to train the model.

    regr = MLPClassifier(random_state=1, max_iter=500, batch_size = args.batch_size, learning_rate_init = args.lr, solver='lbfgs').fit(train_X, train_y)
    regr.get_params()

    # Print the coefficients of the trained classifier, and save the coefficients
    joblib.dump(regr, os.path.join(args.model_dir, "model.joblib")) 
    
    return regr
    
def accuracy(y_pred, y_true):
    
    cm = confusion_matrix(y_pred, y_true)
    
    diagonal_sum = cm.trace()
    sum_of_all_elements = cm.sum()
    
    rt = diagonal_sum / sum_of_all_elements
    
    print ('Accuracy: {}'.format(rt))
    
    return rt
    
    
def test(regr, args):
    test_data=pd.read_csv(os.path.join(args.test, 'test_set.csv'), engine='python')

    # labels are in the first column
    y_true = test_data['truth']
    test_x = test_data[test_data.columns[1:len(test_data)]]
    
    y_pred = regr.predict(test_x)
    
    accuracy(y_pred, y_true)
    
if __name__ == '__main__':

    args = parse_args()
    
    regr = train(args)
    
    test(regr, args)

Overwriting train.py


In [12]:
from sagemaker.sklearn.model import SKLearnModel

role = sagemaker.get_execution_role()

model = SKLearnModel(model_data = model_data,
                     role = role, 
                     framework_version = '0.20.0', 
                     py_version='py3',
                     entry_point = 'train.py')

### 3. Create an Endpoint on SageMaker
Now, here comes the complex maneuver. Kidding, it's dirt simple. Let's turn your model into a RESTful API!

In [13]:
predictor = model.deploy(1, 'ml.m4.2xlarge')

---------------!

In [14]:
import sagemaker

from sagemaker.sklearn.model import SKLearnPredictor
sess = sagemaker.Session()

# optional. If your kernel times out, or your need to refresh, here's how you can easily point to an existing endpoint
endpoint_name = 'sagemaker-scikit-learn-2020-10-14-15-12-50-644'

predictor = SKLearnPredictor(endpoint_name = endpoint_name, sagemaker_session = sess)

Now let's get some predictions from that endpoint. 

In [29]:
test_set = pd.read_csv('test_set.csv')

In [31]:
y_true = test_set['truth']

test_set.drop('truth', inplace=True, axis=1)

In [33]:
import pandas as pd

y_pred = pd.DataFrame(predictor.predict(test_set))

assert len(y_pred) == test_set.shape[0]

### 4. Enable Autoscaling on your Endpoint
For the sake of argument, let's say we're happy with this model and want to continue supporting it in prod. Our next step might be to enable autoscaling. Let's do that right here.

In [32]:
import boto3

def get_resource_id(endpoint_name):

    client = boto3.client('sagemaker')

    response = client.describe_endpoint(
        EndpointName=endpoint_name)

    variant_name = response['ProductionVariants'][0]['VariantName']
    resource_id = 'endpoint/{}/variant/{}'.format(endpoint_name, variant_name)
    
    return resource_id

resource_id = get_resource_id(endpoint_name)

In [19]:
import boto3

role = sagemaker.get_execution_role()

def set_scaling_policy(resource_id, min_capacity = 1, max_capacity = 8, role = role):

    scaling_client = boto3.client('application-autoscaling')

    response = scaling_client.register_scalable_target(
        ServiceNamespace='sagemaker',
        ResourceId=resource_id,
        ScalableDimension='sagemaker:variant:DesiredInstanceCount',
        MinCapacity=min_capacity,
        MaxCapacity=max_capacity,
        RoleARN=role)
    
    return response

res = set_scaling_policy(resource_id)

### 5. Enable Model Monitor on your Endpoint
Now that you have a model up and running, with autoscaling enabled, let's set up model monitor on that endpoint. 

In [20]:
import sagemaker
import os

sess = sagemaker.Session()

bucket = sess.default_bucket()

prefix = 'model-hosting'

s3_capture_upload_path =  's3://{}/{}/model-monitor'.format(bucket, prefix)

print ('about to set up monitoring for endpoint named {}'.format(endpoint_name))

about to set up monitoring for endpoint named sagemaker-scikit-learn-2020-10-14-15-12-50-644


Now, let's set up a data capture config.

In [21]:
from sagemaker.model_monitor import DataCaptureConfig

data_capture_config = DataCaptureConfig(
                        enable_capture = True,
                        sampling_percentage=50,
                        destination_s3_uri=s3_capture_upload_path,
                        capture_options=["REQUEST", "RESPONSE"],
                        csv_content_types=["text/csv"],
                        json_content_types=["application/json"])

# Now it is time to apply the new configuration and wait for it to be applied
predictor.update_data_capture_config(data_capture_config=data_capture_config)

sess.wait_for_endpoint(endpoint=endpoint_name)

---------------!!

{'EndpointName': 'sagemaker-scikit-learn-2020-10-14-15-12-50-644',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:181880743555:endpoint/sagemaker-scikit-learn-2020-10-14-15-12-50-644',
 'EndpointConfigName': 'sagemaker-scikit-learn-2020-10-14-15-12-2020-10-14-15-23-50-458',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'DeployedImages': [{'SpecifiedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3',
     'ResolvedImage': '683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-scikit-learn@sha256:1476311657fe444be8f95f472a8a91a3da577d558e1046678ee76c611f7132ca',
     'ResolutionTime': datetime.datetime(2020, 10, 14, 15, 23, 53, 899000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1}],
 'DataCaptureConfig': {'EnableCapture': True,
  'CaptureStatus': 'Started',
  'CurrentSamplingPercentage': 50,
  'DestinationS3Uri': 's3://sagemaker-us-east-1-181880743555/m

Next step here is to pass in our training data, and ask SageMaker to learn baseline thresholds for all of our features. 

First, let's make sure the data we used to train our model is stored in S3.

In [34]:
msg = 'aws s3 cp train_set.csv s3://{}/{}/train/'.format(bucket, prefix)
os.system(msg)

0

In [35]:
# todo - show them how to get access to this training data
s3_training_data_path = 's3://{}/{}/train/train_set.csv'.format(bucket, prefix)

s3_baseline_results =  's3://{}/{}/model-monitor/baseline-results'.format(bucket, prefix)


In [36]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

my_default_monitor.suggest_baseline(
    baseline_dataset=s3_training_data_path,
    
    # change header to false if not included
    dataset_format=DatasetFormat.csv(header=False),
    output_s3_uri=s3_baseline_results,
    wait=True
)


Job Name:  baseline-suggestion-job-2020-10-14-15-46-58-310
Inputs:  [{'InputName': 'baseline_dataset_input', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-181880743555/model-hosting/train/train_set.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-181880743555/model-hosting/model-monitor/baseline-results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
........................2020-10-14 15:50:44,484 - __main__ - INFO - All params:{'ProcessingJobArn': 'arn:aws:sagemaker:us-east-1:181880743555:processing-job/baseline-suggestion-job-2020-10-14-15-46-58-310', 'ProcessingJobName': 'baseline-suggestion-job-2020-10-14-15-46-58-310', 'Environment': {'dataset_format': '{"csv": {"header": false, "output_columns_position": "START"}}',

If you like, you can download the results from S3 and analyze. In the interest of time, we'll move on to setting up the monitoring schedule. 

In [37]:
from sagemaker.model_monitor import CronExpressionGenerator
from time import gmtime, strftime

mon_schedule_name = 'bi-hourly'
s3_report_path = 's3://{}/{}/model-monitor/monitoring-job-results'.format(bucket, prefix)

my_default_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=endpoint_name,
    output_s3_uri=s3_report_path,
    statistics=my_default_monitor.baseline_statistics(),
    constraints=my_default_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.daily(),
    enable_cloudwatch_metrics=True)


Creating Monitoring Schedule with name: bi-hourly


---
# Tune your model and re-deploy onto the SageMaker Endpoint

Alright, we made it pretty far already! Now that we have monitoring enabled on this endpoint, let's imagine that something goes awry. We realize that we need a new model hosted on this RESTful API. How are we going to do that?

First, let's go about getting a new model. Given that the dataset here is pretty small, less than even 500 rows on the training set, why not try out AutoGluon? AutoGluon is a competitive choice here because it will actually augment our data for us. Said another way, Autogluon will make our original dataset larger by using Transformers and masking columns. Pretty cool!

In [13]:
!mkdir src

In [33]:
%%writefile src/requirements.txt

autogluon
sagemaker
awscli 
boto3
PrettyTable
bokeh
numpy==1.16.1
matplotlib
sagemaker-experiments

Overwriting src/requirements.txt


In [34]:
%%writefile src/train.py

import ast
import argparse
import logging
import warnings
import os
import json
import glob
import subprocess
import sys
import boto3
import pickle
import pandas as pd
from collections import Counter
from timeit import default_timer as timer
import time

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent
from smexperiments.tracker import Tracker

sys.path.insert(0, 'package')
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)
    from prettytable import PrettyTable
    import autogluon as ag
    from autogluon import TabularPrediction as task
    from autogluon.task.tabular_prediction import TabularDataset
    
# ------------------------------------------------------------ #
# Training methods                                             #
# ------------------------------------------------------------ #

def du(path):
    """disk usage in human readable format (e.g. '2,1GB')"""
    return subprocess.check_output(['du','-sh', path]).split()[0].decode('utf-8')

def __load_input_data(path: str) -> TabularDataset:
    """
    Load training data as dataframe
    :param path:
    :return: DataFrame
    """
    input_data_files = os.listdir(path)
    try:
        input_dfs = [pd.read_csv(f'{path}/{data_file}') for data_file in input_data_files]
        return task.Dataset(df=pd.concat(input_dfs))
    except:
        print(f'No csv data in {path}!')
        return None

def train(args):
        
    is_distributed = len(args.hosts) > 1
    host_rank = args.hosts.index(args.current_host)    
    dist_ip_addrs = args.hosts
    dist_ip_addrs.pop(host_rank)
    ngpus_per_trial = 1 if args.num_gpus > 0 else 0

    # load training and validation data
    print(f'Train files: {os.listdir(args.train)}')
    train_data = __load_input_data(args.train)
    print(f'Label counts: {dict(Counter(train_data[args.label]))}')
    
    predictor = task.fit(
        train_data=train_data,
        label=args.label,            
        output_directory=args.model_dir,
        problem_type=args.problem_type,
        eval_metric=args.eval_metric,
        stopping_metric=args.stopping_metric,
        auto_stack=args.auto_stack, # default: False
        hyperparameter_tune=args.hyperparameter_tune, # default: False
        feature_prune=args.feature_prune, # default: False
        holdout_frac=args.holdout_frac, # default: None
        num_bagging_folds=args.num_bagging_folds, # default: 0
        num_bagging_sets=args.num_bagging_sets, # default: None
        stack_ensemble_levels=args.stack_ensemble_levels, # default: 0
        cache_data=args.cache_data,
        time_limits=args.time_limits,
        num_trials=args.num_trials, # default: None
        search_strategy=args.search_strategy, # default: 'random'
        search_options=args.search_options,
        visualizer=args.visualizer,
        verbosity=args.verbosity
    )
    
    # Results summary
    predictor.fit_summary(verbosity=1)

    # Leaderboard on optional test data
    if args.test:
        print(f'Test files: {os.listdir(args.test)}')
        test_data = __load_input_data(args.test)    
        print('Running model on test data and getting Leaderboard...')
        leaderboard = predictor.leaderboard(dataset=test_data, silent=True)
        def format_for_print(df):
            table = PrettyTable(list(df.columns))
            for row in df.itertuples():
                table.add_row(row[1:])
            return str(table)
        print(format_for_print(leaderboard), end='\n\n')

    # Files summary
    print(f'Model export summary:')
    print(f"/opt/ml/model/: {os.listdir('/opt/ml/model/')}")
    models_contents = os.listdir('/opt/ml/model/models')
    print(f"/opt/ml/model/models: {models_contents}")
    print(f"/opt/ml/model directory size: {du('/opt/ml/model/')}\n")

# ------------------------------------------------------------ #
# Training execution                                           #
# ------------------------------------------------------------ #

def str2bool(v):
    return v.lower() in ('yes', 'true', 't', '1')

def parse_args():

    parser = argparse.ArgumentParser(
             formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.register('type','bool',str2bool) # add type keyword to registries

    parser.add_argument('--hosts', type=list, default=json.loads(os.environ['SM_HOSTS']))    
    parser.add_argument('--current-host', type=str, default=os.environ['SM_CURRENT_HOST'])
    parser.add_argument('--num-gpus', type=int, default=os.environ['SM_NUM_GPUS'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR']) # /opt/ml/model
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAINING'])
    parser.add_argument('--test', type=str, default='') # /opt/ml/input/data/test
    parser.add_argument('--label', type=str, default='truth',
                        help="Name of the column that contains the target variable to predict.")
    
    parser.add_argument('--problem_type', type=str, default=None,
                        help=("Type of prediction problem, i.e. is this a binary/multiclass classification or "
                              "regression problem options: 'binary', 'multiclass', 'regression'). "
                              "If `problem_type = None`, the prediction problem type is inferred based "
                              "on the label-values in provided dataset."))
    parser.add_argument('--eval_metric', type=str, default=None,
                        help=("Metric by which predictions will be ultimately evaluated on test data."
                              "AutoGluon tunes factors such as hyperparameters, early-stopping, ensemble-weights, etc. "
                              "in order to improve this metric on validation data. "
                              "If `eval_metric = None`, it is automatically chosen based on `problem_type`. "
                              "Defaults to 'accuracy' for binary and multiclass classification and "
                              "'root_mean_squared_error' for regression. "
                              "Otherwise, options for classification: [ "
                              "    'accuracy', 'balanced_accuracy', 'f1', 'f1_macro', 'f1_micro', 'f1_weighted', "
                              "    'roc_auc', 'average_precision', 'precision', 'precision_macro', 'precision_micro', 'precision_weighted', "
                              "    'recall', 'recall_macro', 'recall_micro', 'recall_weighted', 'log_loss', 'pac_score']. "
                              "Options for regression: ['root_mean_squared_error', 'mean_squared_error', "
                              "'mean_absolute_error', 'median_absolute_error', 'r2']. "
                              "For more information on these options, see `sklearn.metrics`: "
                              "https://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics "
                              "You can also pass your own evaluation function here as long as it follows formatting of the functions "
                              "defined in `autogluon/utils/tabular/metrics/`. "))
    parser.add_argument('--stopping_metric', type=str, default=None,
                        help=("Metric which models use to early stop to avoid overfitting. "
                              "`stopping_metric` is not used by weighted ensembles, instead weighted ensembles maximize `eval_metric`. "
                              "Defaults to `eval_metric` value except when `eval_metric='roc_auc'`, where it defaults to `log_loss`."))      
    parser.add_argument('--auto_stack', type='bool', default=False,
                        help=("Whether to have AutoGluon automatically attempt to select optimal "
                              "num_bagging_folds and stack_ensemble_levels based on data properties. "
                              "Note: Overrides num_bagging_folds and stack_ensemble_levels values. "
                              "Note: This can increase training time by up to 20x, but can produce much better results. "
                              "Note: This can increase inference time by up to 20x."))
    parser.add_argument('--hyperparameter_tune', type='bool', default=False,
                        help=("Whether to tune hyperparameters or just use fixed hyperparameter values "
                              "for each model. Setting as True will increase `fit()` runtimes."))
    parser.add_argument('--feature_prune', type='bool', default=False,
                        help="Whether or not to perform feature selection.")
    parser.add_argument('--holdout_frac', type=float, default=None, 
                        help=("Fraction of train_data to holdout as tuning data for optimizing hyperparameters "
                              "(ignored unless `tuning_data = None`, ignored if `num_bagging_folds != 0`). "
                              "Default value is selected based on the number of rows in the training data. "
                              "Default values range from 0.2 at 2,500 rows to 0.01 at 250,000 rows. "
                              "Default value is doubled if `hyperparameter_tune = True`, up to a maximum of 0.2. "
                              "Disabled if `num_bagging_folds >= 2`."))    
    parser.add_argument('--num_bagging_folds', type=int, default=0, 
                        help=("Number of folds used for bagging of models. When `num_bagging_folds = k`, "
                              "training time is roughly increased by a factor of `k` (set = 0 to disable bagging). "
                              "Disabled by default, but we recommend values between 5-10 to maximize predictive performance. "
                              "Increasing num_bagging_folds will result in models with lower bias but that are more prone to overfitting. "
                              "Values > 10 may produce diminishing returns, and can even harm overall results due to overfitting. "
                              "To further improve predictions, avoid increasing num_bagging_folds much beyond 10 "
                              "and instead increase num_bagging_sets. "))    
    parser.add_argument('--num_bagging_sets', type=int, default=None,
                        help=("Number of repeats of kfold bagging to perform (values must be >= 1). "
                              "Total number of models trained during bagging = num_bagging_folds * num_bagging_sets. "
                              "Defaults to 1 if time_limits is not specified, otherwise 20 "
                              "(always disabled if num_bagging_folds is not specified). "
                              "Values greater than 1 will result in superior predictive performance, "
                              "especially on smaller problems and with stacking enabled. "
                              "Increasing num_bagged_sets reduces the bagged aggregated variance without "
                              "increasing the amount each model is overfit."))
    parser.add_argument('--stack_ensemble_levels', type=int, default=0, 
                        help=("Number of stacking levels to use in stack ensemble. "
                              "Roughly increases model training time by factor of `stack_ensemble_levels+1` " 
                              "(set = 0 to disable stack ensembling).  "
                              "Disabled by default, but we recommend values between 1-3 to maximize predictive performance. "
                              "To prevent overfitting, this argument is ignored unless you have also set `num_bagging_folds >= 2`."))
    parser.add_argument('--hyperparameters', type=lambda s: ast.literal_eval(s), default=None,
                        help="Refer to docs: https://autogluon.mxnet.io/api/autogluon.task.html")
    parser.add_argument('--cache_data', type='bool', default=True,
                       help=("Whether the predictor returned by this `fit()` call should be able to be further trained "
                             "via another future `fit()` call. "
                             "When enabled, the training and validation data are saved to disk for future reuse."))
    parser.add_argument('--time_limits', type=int, default=None, 
                        help=("Approximately how long `fit()` should run for (wallclock time in seconds)."
                              "If not specified, `fit()` will run until all models have completed training, "
                              "but will not repeatedly bag models unless `num_bagging_sets` is specified."))
    parser.add_argument('--num_trials', type=int, default=None, 
                        help=("Maximal number of different hyperparameter settings of each "
                              "model type to evaluate during HPO. (only matters if "
                              "hyperparameter_tune = True). If both `time_limits` and "
                              "`num_trials` are specified, `time_limits` takes precedent."))    
    parser.add_argument('--search_strategy', type=str, default='random',
                        help=("Which hyperparameter search algorithm to use. "
                              "Options include: 'random' (random search), 'skopt' "
                              "(SKopt Bayesian optimization), 'grid' (grid search), "
                              "'hyperband' (Hyperband), 'rl' (reinforcement learner)"))      
    parser.add_argument('--search_options', type=lambda s: ast.literal_eval(s), default=None,
                        help="Auxiliary keyword arguments to pass to the searcher that performs hyperparameter optimization.")
    parser.add_argument('--nthreads_per_trial', type=int, default=None,
                        help="How many CPUs to use in each training run of an individual model. This is automatically determined by AutoGluon when left as None (based on available compute).")
    parser.add_argument('--ngpus_per_trial', type=int, default=None,
                        help="How many GPUs to use in each trial (ie. single training run of a model). This is automatically determined by AutoGluon when left as None.")
    parser.add_argument('--dist_ip_addrs', type=list, default=None,
                        help="List of IP addresses corresponding to remote workers, in order to leverage distributed computation.") 
    parser.add_argument('--visualizer', type=str, default='none',
                        help=("How to visualize the neural network training progress during `fit()`. "
                              "Options: ['mxboard', 'tensorboard', 'none']."))          
    parser.add_argument('--verbosity', type=int, default=2, 
                        help=("Verbosity levels range from 0 to 4 and control how much information is printed during fit(). "
                              "Higher levels correspond to more detailed print statements (you can set verbosity = 0 to suppress warnings). "
                              "If using logging, you can alternatively control amount of information printed via `logger.setLevel(L)`, "
                              "where `L` ranges from 0 to 50 (Note: higher values of `L` correspond to fewer print statements, "
                              "opposite of verbosity levels"))
    parser.add_argument('--debug', type='bool', default=False,
                       help=("Whether to set logging level to DEBUG"))  
        
    parser.add_argument('--feature_importance', type='bool', default=True)

    return parser.parse_args()


def set_experiment_config(experiment_basename = None):
    '''
    Optionally takes an base name for the experiment. Has a hard dependency on boto3 installation. 
    Creates a new experiment using the basename, otherwise simply uses autogluon as basename.
    May run into issues on Experiments' requirements for basename config downstream.
    '''
    now = int(time.time())
    
    if experiment_basename:
        experiment_name = '{}-autogluon-{}'.format(experiment_basename, now)
    else:
        experiment_name = 'autogluon-{}'.format(now)
    
    try:
        client = boto3.Session().client('sagemaker')
    except:
        print ('You need to install boto3 to create an experiment. Try pip install --upgrade boto3')
        return ''
    
    try:
        Experiment.create(experiment_name=experiment_name, 
                            description="Running AutoGluon Tabular with SageMaker Experiments", 
                            sagemaker_boto_client=client)
        print ('Created an experiment named {}, you should be able to see this in SageMaker Studio right now.'.format(experiment_name))
        
    except:
        print ('Could not create the experiment. Is your basename properly configured? Also try installing the sagemaker experiments SDK with pip install sagemaker-experiments.')
        return ''
    
    return experiment_name

if __name__ == "__main__":
    start = timer()

    args = parse_args()
        
    # Print SageMaker args
    print('\n====== args ======')
    for k,v in vars(args).items():
        print(f'{k},  type: {type(v)},  value: {v}')
    print()
    
    train()

    # Package inference code with model export
    subprocess.call('mkdir /opt/ml/model/code'.split())
    subprocess.call('cp /opt/ml/code/inference.py /opt/ml/model/code/'.split())
    
    elapsed_time = round(timer()-start,3)
    print(f'Elapsed time: {elapsed_time} seconds')  
    print('===== Training Completed =====')

Overwriting src/train.py


In [32]:
from sagemaker.mxnet.estimator import MXNet
from sagemaker import get_execution_role

role = get_execution_role()

estimator = MXNet(source_dir = 'src',
                    entry_point = 'train.py',
                    role=role,
                    framework_version = '1.7.0',
                    py_version = 'py3',
                    instance_count=1,
                    instance_type='ml.m5.2xlarge',
                    volume_size=100)         

s3_path = 's3://sagemaker-us-east-1-181880743555/model-hosting/test_set.csv'

estimator.fit(s3_path, wait=False)

In [4]:
# from sagemaker.sklearn.estimator import SKLearn
# from sagemaker import get_execution_role

# script_path = 'train.py'

# # first, let's get the estimator defined 
# est = SKLearn(entry_point=script_path,
#                 instance_type="ml.c4.xlarge",
#                 instance_count = 1,
#                 role=role,
#                 sagemaker_session=sess,
#                 py_version = 'py3',
#                 framework_version = '0.20.0')

# # then, let's set up the tuning framework 
# from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

# hyperparameter_ranges = {'lr': ContinuousParameter(0.00001, 0.001),
#                          'batch-size': IntegerParameter(25, 300)}

In [5]:
# objective_metric_name = 'Accuracy'
# objective_type = 'Maximize'
# metric_definitions = [{'Name': 'Accuracy',
#                        'Regex': 'Accuracy: ([0-9\\.]+)'}]

In [6]:
# tuner = HyperparameterTuner(est,
#                             objective_metric_name,
#                             hyperparameter_ranges,
#                             metric_definitions,
#                             max_jobs=20,
#                             max_parallel_jobs=3,
#                             objective_type=objective_type)

In [7]:
# msg = 'aws s3 cp test_set.csv s3://{}/{}/ && aws s3 cp train_set.csv s3://{}/{}/'.format(bucket, prefix, bucket, prefix)
# os.system(msg)

In [8]:
# # may complain about not wanting headers 
# inputs = {'train': 's3://{}/{}/train_set.csv'.format(bucket, prefix),
#           'test': 's3://{}/{}/test_set.csv'.format(bucket, prefix)}

In [9]:
# tuner.fit(inputs)

### Redeploy to existing SageMaker Endpoint

In [2]:
from sagemaker.tuner import HyperparameterTuner

job_name = 'sagemaker-scikit-lea-201014-1830'

tuner = HyperparameterTuner.attach(job_name)


In [4]:
best_estimator = tuner.best_estimator()


2020-10-14 18:33:33 Starting - Preparing the instances for training
2020-10-14 18:33:33 Downloading - Downloading input data
2020-10-14 18:33:33 Training - Training image download completed. Training in progress.
2020-10-14 18:33:33 Uploading - Uploading generated training model
2020-10-14 18:33:33 Completed - Training job completed


In [12]:
model = best_estimator.create_model()
model_name = model.name

In [40]:
import boto3
import random
import time
import datetime

def create_model(model, now):
    sm_client = boto3.client('sagemaker')
    
    x = random.randint(1, 100)
    
    model_name = '{}-{}'.format(model.name, now)
    
    response = sm_client.create_model(ModelName=model_name,
            PrimaryContainer={'ContainerHostname': 'string','Image': model.image_uri, 'ModelDataUrl': model.model_data},
            ExecutionRoleArn= 'arn:aws:iam::181880743555:role/service-role/AmazonSageMaker-ExecutionRole-20200929T125134')

    return response

def get_endpoint_config(model_name, now):
    
    sm_client = boto3.client('sagemaker')

    endpoint_config_name = 'ec-{}-{}'.format(model_name, now)
    
    response = sm_client.create_endpoint_config(EndpointConfigName= endpoint_config_name,
                                            ProductionVariants=[{'VariantName': 'v-{}'.format(model_name),
                                                                    'ModelName': model_name,
                                                                    'InitialInstanceCount': 1,
                                                                    'InstanceType':'ml.m5.large'}])
    return endpoint_config_name

def update_endpoint(model_name, endpoint_name, now):
    
    sm_client = boto3.client('sagemaker')

    endpoint_config = get_endpoint_config(model_name, now)
    
    # deregister a scaling policy 
    resource_id = get_resource_id(endpoint_name)
    
    client = boto3.client('application-autoscaling')
    
    
    try:
        response = client.deregister_scalable_target(ServiceNamespace='sagemaker',
                                                    ResourceId=resource_id,
                                                    ScalableDimension='sagemaker:variant:DesiredInstanceCount')
        
    except:
        print ('no autoscaling policy to deregister, continuing')
    # get monitoring schedules
    
    try:
        response = sm_client.list_monitoring_schedules(EndpointName=endpoint_name,
                                                    MaxResults=10,
                                                    StatusEquals='Scheduled')
        # delete monitoring schedules 
        for each in response['MonitoringScheduleSummaries']:
            name = each['MonitoringScheduleName']
            response = sm_client.delete_monitoring_schedule(MonitoringScheduleName=name)
            
    except:
        print ('already deleted the monitoring schedules')
    
    response = sm_client.update_endpoint(EndpointName=endpoint_name,
                                        EndpointConfigName=endpoint_config)
    
    return response

                                
now = str(datetime.datetime.now()).split('.')[-1]
                                
endpoint_name = 'sagemaker-scikit-learn-2020-10-14-15-12-50-644'

create_model(model, now)

update_endpoint(model_name, endpoint_name, now)

no autoscaling policy to deregister, continuing


{'EndpointArn': 'arn:aws:sagemaker:us-east-1:181880743555:endpoint/sagemaker-scikit-learn-2020-10-14-15-12-50-644',
 'ResponseMetadata': {'RequestId': '568a69ab-fc94-4812-9b4e-9e8575063b72',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '568a69ab-fc94-4812-9b4e-9e8575063b72',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '114',
   'date': 'Wed, 14 Oct 2020 21:01:15 GMT'},
  'RetryAttempts': 0}}

---
# Automate with Notebook Runner
Now we're able to monitor new endpoints, we want the ability to automate this whole flow so that we can do it rapidly. As it so happens, a simple and fast way of doing that is using SageMaker processing jobs, CloudWatch, and Lambda. Luckily we can import all of the infrastructure we need using a simple toolkit, which we'll step through here.

GitHub notes are right here: https://github.com/aws-samples/sagemaker-run-notebook

In [112]:
# todo - make sure they have the right execution role here, add cfn all access, then a trust relationship, then inlines to allow create stack, plus codebuild create project nad start build 

In [ ]:
# !wget https://github.com/aws-samples/sagemaker-run-notebook/releases/download/v0.15.0/sagemaker_run_notebook-0.15.0.tar.gz

In [ ]:
# !pip install sagemaker_run_notebook-0.15.0.tar.gz

In [ ]:
# !run-notebook create-infrastructure --update

In [117]:
%%writefile requirements.txt
awscli
boto3
sagemaker
pandas
sklearn

Writing requirements.txt


In [ ]:
# !run-notebook create-container --requirements requirements.txt

In [ ]:
# !wget https://github.com/aws-samples/sagemaker-run-notebook/releases/download/v0.15.0/install-run-notebook.sh

Next, __you need to open a system terminal on Studio, cd into the directory where we just downloaded `install-run-notebook.sh`, and run the command `bash install-run-notebook.sh`.__ This will run for a few minutes, then prompt you to refresh your web browser. Do that, and you'll see a new Jupyter Widget!

After restarting your Studio page, click on the spaceship widget on the top lefthand side of your Stuio domain view. Make sure you're actually looking at an ipython notebook while you do this.

Using the widget is super simple. Paste in your execution role, which you can find by running `sagemaker.get_execution_role()` locally. Then paste in your ECR image repository, which you can find by opening up the ECR page in the AWS console. It should default to `notebook-runner`, so you can just paste that in directly.

Then click the big blue `run now` button, and __this entire notebook is going to run on a SageMaker processing job.__ 

Before you do that, you'll want to comment-out those last few cells you ran to install this toolkit and get the infrastructure up and running. 

If you want, you can parameterize this entire notebook using Papermill. Read more about how to do that with the following resources:
- Blog post: https://aws.amazon.com/blogs/machine-learning/scheduling-jupyter-notebooks-on-sagemaker-ephemeral-instances/
- GitHub repository: https://github.com/aws-samples/sagemaker-run-notebook